In [1]:
!pip3 install kafka-python


  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [2]:
from pymongo import MongoClient
from kafka import KafkaProducer
from kafka import KafkaConsumer
from json import dumps
import json
import pandas as pd
from scraping import get_soup,get_table

In [3]:
link = "https://fbref.com/en/comps/9/shooting/Premier-League-Stats"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

soup = get_soup(link, headers)
df=get_table(soup)
df


,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Gls,Sh,...,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches,PlayerID
0,1,Max Aarons,eng ENG,DF,Bournemouth,23-342,2000,10.6,0,1,...,0,0,0,0.0,0.0,0.02,0.0,0.0,Matches,774cf58b
1,2,Bénie Adama Traore,ci CIV,"FW,MF",Sheffield Utd,21-012,2002,4.3,0,1,...,0,0,0,0.3,0.3,0.27,-0.3,-0.3,Matches,77816c91
2,3,Tosin Adarabioyo,eng ENG,DF,Fulham,26-079,1997,2.0,1,1,...,0,0,0,0.0,0.0,0.03,+1.0,+1.0,Matches,c81d773d
3,4,Elijah Adebayo,eng ENG,FW,Luton Town,25-339,1998,6.5,4,11,...,0,0,0,1.9,1.9,0.17,+2.1,+2.1,Matches,0480ebc7
4,5,Simon Adingra,ci CIV,"FW,MF",Brighton,21-345,2002,11.3,4,19,...,0,0,0,2.3,2.3,0.12,+1.7,+1.7,Matches,4dcec659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,497,Anass Zaroury,ma MAR,"FW,MF",Burnley,23-035,2000,1.5,0,3,...,0,0,0,0.2,0.2,0.06,-0.2,-0.2,Matches,NaN
516,498,Oleksandr Zinchenko,ua UKR,DF,Arsenal,26-362,1996,10.7,1,11,...,0,0,0,0.4,0.4,0.03,+0.6,+0.6,Matches,NaN
517,499,Kurt Zouma,fr FRA,DF,West Ham,29-046,1994,13.7,1,8,...,0,0,0,0.9,0.9,0.12,+0.1,+0.1,Matches,NaN
518,500,Martin Ødegaard,no NOR,MF,Arsenal,24-360,1998,12.7,4,26,...,0,2,2,3.2,1.7,0.06,+0.8,+0.3,Matches,NaN


In [17]:
producer = KafkaProducer(bootstrap_servers=[':9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))


In [10]:
for i, row in df.iterrows():
    message = row.to_dict()
    producer.send('shooting', value=message)

        

Rk                    1
Player       Max Aarons
Nation          eng ENG
Pos                  DF
Squad       Bournemouth
Age              23-342
Born               2000
90s                10.6
Gls                   0
Sh                    1
SoT                   0
SoT%                0.0
Sh/90              0.09
SoT/90             0.00
G/Sh               0.00
G/SoT               NaN
Dist               23.9
FK                    0
PK                    0
PKatt                 0
xG                  0.0
npxG                0.0
npxG/Sh            0.02
G-xG                0.0
np:G-xG             0.0
Matches         Matches
PlayerID       774cf58b
Name: 0, dtype: object
<class 'pandas.core.series.Series'>



{'Rk': '1', 'Player': 'Max Aarons', 'Nation': 'eng ENG', 'Pos': 'DF', 'Squad': 'Bournemouth', 'Age': '23-342', 'Born': '2000', '90s': '10.6', 'Gls': '0', 'Sh': '1', 'SoT': '0', 'SoT%': '0.0', 'Sh/90': '0.09', 'SoT/90': '0.00', 'G/Sh': '0.00', 'G/SoT': nan, 'Dist': '23.9', 'FK': '0', 'PK': 

In [19]:
bootstrap_servers = ['localhost:9092']
topic = 'shooting'

In [20]:
mongo_uri = 'mongodb://localhost:27017/'
mongo_db = 'Premier_League_Stats'
mongo_collection = 'shooting'

# Connexion à MongoDB
client = MongoClient(mongo_uri)
db = client[mongo_db]
collection = db[mongo_collection]

In [ ]:
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',
    group_id=None,
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

for message in consumer:
    data = message.value
    print(message)
    # Insérer les données dans MongoDB
    collection.insert_one(data)

ConsumerRecord(topic='shooting', partition=0, offset=0, timestamp=1702320091174, timestamp_type=0, key=None, value={'Rk': '1', 'Player': 'Max Aarons', 'Nation': 'eng ENG', 'Pos': 'DF', 'Squad': 'Bournemouth', 'Age': '23-341', 'Born': '2000', '90s': '10.6', 'Gls': '0', 'Sh': '1', 'SoT': '0', 'SoT%': '0.0', 'Sh/90': '0.09', 'SoT/90': '0.00', 'G/Sh': '0.00', 'G/SoT': nan, 'Dist': '23.9', 'FK': '0', 'PK': '0', 'PKatt': '0', 'xG': '0.0', 'npxG': '0.0', 'npxG/Sh': '0.02', 'G-xG': '0.0', 'np:G-xG': '0.0', 'Matches': 'Matches', 'PlayerID': '774cf58b'}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=434, serialized_header_size=-1)
ConsumerRecord(topic='shooting', partition=0, offset=1, timestamp=1702320091174, timestamp_type=0, key=None, value={'Rk': '2', 'Player': 'Bénie Adama Traore', 'Nation': 'ci CIV', 'Pos': 'FW,MF', 'Squad': 'Sheffield Utd', 'Age': '21-011', 'Born': '2002', '90s': '4.3', 'Gls': '0', 'Sh': '1', 'SoT': '1', 'SoT%': '100.0', 'Sh/90': '0.23', 'SoT/90

ConsumerRecord(topic='shooting', partition=0, offset=421, timestamp=1702320091347, timestamp_type=0, key=None, value={'Rk': '406', 'Player': 'Davinson Sánchez', 'Nation': 'co COL', 'Pos': 'DF', 'Squad': 'Tottenham', 'Age': '27-182', 'Born': '1996', '90s': '0.9', 'Gls': '0', 'Sh': '1', 'SoT': '0', 'SoT%': '0.0', 'Sh/90': '1.17', 'SoT/90': '0.00', 'G/Sh': '0.00', 'G/SoT': nan, 'Dist': '8.6', 'FK': '0', 'PK': '0', 'PKatt': '0', 'xG': '0.1', 'npxG': '0.1', 'npxG/Sh': '0.12', 'G-xG': '-0.1', 'np:G-xG': '-0.1', 'Matches': 'Matches', 'PlayerID': nan}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=437, serialized_header_size=-1)
ConsumerRecord(topic='shooting', partition=0, offset=422, timestamp=1702320091347, timestamp_type=0, key=None, value={'Rk': '407', 'Player': 'Robert Sánchez', 'Nation': 'es ESP', 'Pos': 'GK', 'Squad': 'Chelsea', 'Age': '26-023', 'Born': '1997', '90s': '15.9', 'Gls': '0', 'Sh': '0', 'SoT': '0', 'SoT%': nan, 'Sh/90': '0.00', 'SoT/90': '0.00', '

ConsumerRecord(topic='shooting', partition=0, offset=887, timestamp=1702320132266, timestamp_type=0, key=None, value={'Rk': '354', 'Player': 'Cole Palmer', 'Nation': 'eng ENG', 'Pos': 'MF', 'Squad': 'Manchester City', 'Age': '21-219', 'Born': '2002', '90s': '0.1', 'Gls': '0', 'Sh': '0', 'SoT': '0', 'SoT%': nan, 'Sh/90': '0.00', 'SoT/90': '0.00', 'G/Sh': nan, 'G/SoT': nan, 'Dist': nan, 'FK': '0', 'PK': '0', 'PKatt': '0', 'xG': '0.0', 'npxG': '0.0', 'npxG/Sh': nan, 'G-xG': '0.0', 'np:G-xG': '0.0', 'Matches': 'Matches', 'PlayerID': nan}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=422, serialized_header_size=-1)
ConsumerRecord(topic='shooting', partition=0, offset=888, timestamp=1702320132266, timestamp_type=0, key=None, value={'Rk': '355', 'Player': 'Cole Palmer', 'Nation': 'eng ENG', 'Pos': 'FW,MF', 'Squad': 'Chelsea', 'Age': '21-219', 'Born': '2002', '90s': '9.7', 'Gls': '5', 'Sh': '22', 'SoT': '8', 'SoT%': '36.4', 'Sh/90': '2.27', 'SoT/90': '0.82', 'G/Sh':

ConsumerRecord(topic='shooting', partition=0, offset=1335, timestamp=1702320974232, timestamp_type=0, key=None, value={'Rk': '285', 'Player': 'Neal Maupay', 'Nation': 'fr FRA', 'Pos': 'FW', 'Squad': 'Brentford', 'Age': '27-119', 'Born': '1996', '90s': '6.2', 'Gls': '2', 'Sh': '21', 'SoT': '6', 'SoT%': '28.6', 'Sh/90': '3.41', 'SoT/90': '0.97', 'G/Sh': '0.10', 'G/SoT': '0.33', 'Dist': '14.1', 'FK': '1', 'PK': '0', 'PKatt': '0', 'xG': '3.6', 'npxG': '3.6', 'npxG/Sh': '0.17', 'G-xG': '-1.6', 'np:G-xG': '-1.6', 'Matches': 'Matches', 'PlayerID': nan}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=433, serialized_header_size=-1)
ConsumerRecord(topic='shooting', partition=0, offset=1336, timestamp=1702320974232, timestamp_type=0, key=None, value={'Rk': '286', 'Player': 'Neal Maupay', 'Nation': 'fr FRA', 'Pos': 'FW', 'Squad': 'Everton', 'Age': '27-119', 'Born': '1996', '90s': '1.2', 'Gls': '0', 'Sh': '5', 'SoT': '4', 'SoT%': '80.0', 'Sh/90': '4.05', 'SoT/90': '3.24',

In [60]:
consumer.close()